# Crop Screenshots Notebook

This notebook is designed to crop JPEG images generated from PNG screenshots. 
Using the Python Imaging Library (Pillow), it processes each JPEG image located in a specified input directory, 
applies a predefined bounding box to crop the images, and saves the cropped versions in a designated output directory. 
The cropping is particularly useful for isolating specific parts of the images, such as selections or highlights, 
to focus on relevant content.

## Features:
- Automatically searches for JPEG images in the input directory.
- Crops images based on specified pixel coordinates for a bounding box.
- Saves the cropped images with a modified filename to differentiate them from the originals.
- Produces console output for each cropped image, indicating conversion success.

Ensure that the input directory contains JPEG images before running the notebook, and modify the 
bounding box dimensions as needed to achieve the desired cropping effect.

## Setup:

In [7]:
# Import necessary libraries
import os
from PIL import Image
import matplotlib.pyplot as plt

In [8]:
# Define any bounding boxes to be used

# Bounding box (in pixels)
SQUAD_BATTLES_SELECTION_BBOX = (140, 363, 140 + 290, 363 + 545)  # (left, upper, right, lower)

In [9]:
# Set up input and output directories for image processing
input_dir = '../screenshots/jpg'
output_dir = '../screenshots/jpg/cropped'
os.makedirs(output_dir, exist_ok=True)

### Functions:
Define different cropping functions below.

In [12]:
# Function to crop and resize the image to maintain the aspect ratio of 1920:1080
def crop_and_resize_to_1080p(image_path):
    img = Image.open(image_path)

    original_width, original_height = img.size
    aspect_ratio = 1920 / 1080

    # Calculate new dimensions maintaining the aspect ratio
    if original_width / original_height >= aspect_ratio:
        # Image is wider than 16:9
        new_height = original_height
        new_width = int(original_height * aspect_ratio)
    else:
        # Image is taller than 16:9
        new_width = original_width
        new_height = int(original_width / aspect_ratio)
    
    # Calculate the cropping box to crop from the center
    left = (original_width - new_width) / 2
    top = (original_height - new_height) / 2
    right = (original_width + new_width) / 2
    bottom = (original_height + new_height) / 2

    # Crop the image
    cropped_img = img.crop((left, top, right, bottom))
    
    # Resize the image to 1080p
    resized_img = cropped_img.resize((1920, 1080), Image.LANCZOS)

    return resized_img

# Function to crop based on a predefined bounding box
def crop_with_bbox(image_path, bbox):
    img = Image.open(image_path)
    cropped_img = img.crop(bbox)
    return cropped_img

# Function to save cropped images
def save_image(image, file_path):
    image.save(file_path, 'JPEG', quality=90)
    print(f"Saved: {file_path}")

# Function for batch cropping with bounding box
def batch_crop_with_bbox(input_dir, output_dir, bbox):
    os.makedirs(output_dir, exist_ok=True)
    for file_name in os.listdir(input_dir):
        if file_name.lower().endswith('.jpg'):
            jpg_file_path = os.path.join(input_dir, file_name)
            base_name = os.path.splitext(file_name)[0]
            cropped_file_path = os.path.join(output_dir, f"{base_name}_cropped.jpg")

            cropped_img = crop_with_bbox(jpg_file_path, bbox)
            save_image(cropped_img, cropped_file_path)

# Function for batch processing of images
def batch_crop_and_resize_to_1080p(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for file_name in os.listdir(input_dir):
        if file_name.lower().endswith('.jpg'):
            jpg_file_path = os.path.join(input_dir, file_name)
            base_name = os.path.splitext(file_name)[0]
            cropped_file_path = os.path.join(output_dir, f"{base_name}_cropped.jpg")

            resized_img = crop_and_resize_to_1080p(jpg_file_path)
            save_image(resized_img, cropped_file_path)

## Use Cases:

In [13]:
# 1. Batch crop to centered 1080p
batch_crop_and_resize_to_1080p(input_dir, output_dir)

Saved: ../screenshots/jpg/cropped/new-screenshot1738871871.673785_cropped.jpg
Saved: ../screenshots/jpg/cropped/new-screenshot1738871994.79851_cropped.jpg
Saved: ../screenshots/jpg/cropped/new-screenshot1738871626.05634_cropped.jpg
Saved: ../screenshots/jpg/cropped/new-screenshot1738871609.9618182_cropped.jpg
Saved: ../screenshots/jpg/cropped/new-screenshot1738871632.0353642_cropped.jpg
Saved: ../screenshots/jpg/cropped/new-screenshot1738872150.510043_cropped.jpg
Saved: ../screenshots/jpg/cropped/new-screenshot1738871899.821942_cropped.jpg
Saved: ../screenshots/jpg/cropped/new-screenshot1738872044.329857_cropped.jpg
Saved: ../screenshots/jpg/cropped/new-screenshot1738871599.1558552_cropped.jpg
Saved: ../screenshots/jpg/cropped/new-screenshot1738871914.2988002_cropped.jpg
Saved: ../screenshots/jpg/cropped/new-screenshot1738872164.021826_cropped.jpg
Saved: ../screenshots/jpg/cropped/new-screenshot1738871796.5500212_cropped.jpg
Saved: ../screenshots/jpg/cropped/new-screenshot1738871778.86

In [6]:
# 2. Batch crop using bounding box
#batch_crop_with_bbox(input_dir, output_dir, SQUAD_BATTLES_SELECTION_BBOX)